# Data Merging

In [1]:
import os
import numpy as np
import pandas as pd

In [2]:
%load_ext autoreload
%autoreload 2

## Fetch data from file path

Get the csv file names from a given directory

In [3]:
def get_file_names(directory):
    """returns the csv files in the given string directory path"""
    file_names = []
    for filename in os.listdir(directory):
        if filename.endswith(".csv"):
            file_names.append(os.path.join(directory, filename))
    return file_names

Create a dictionary from the files where the keys are the yearly intervals and the values are the respective dataframes.

In [4]:
def get_dataframes(file_names):
    """Takes in a list of csv file paths.
    Returns a dictionary whose keys are the years
    and values are the corresponding dataframes."""
    df_years = {}
    for file in file_names:
        df_years[file[-29:]] = pd.read_csv(file) # [-29:] Indexes the years.csv eg '201501010000-201601010000.csv'
    return df_years
    

Function to get the path per feature e.g price, load

In [5]:
def get_feature_path(feature_name, main_directory):
    for root, dirs_, files in os.walk(main_directory):
        if feature_name in (root):
            return root

Function to bring it together and return a tuple containing a nested dictionary of the features and the corresponding dictionary of per year dataframes. 

In [6]:
def get_features_df(main_path):
    feats = ['price', 'load']
    feat_dict = {}
    for feat in feats:
        path = get_feature_path(feat, main_path)
        names = get_file_names(path)
        feat_dict[feat] = get_dataframes(names)
        
    return feat_dict[feats[0]], feat_dict[feats[1]]

Used as follows...

In [7]:
path = r'../raw_data/'
price, load = get_features_df(main_path=path) # unpack tuple 

## Create DataFrames from data

### Concatenate dataframes

In [8]:
def concat_dataframes(feat):
    keys = list(feat.keys())
    df = pd.concat([
        feat[keys[0]],  # year 2015
        feat[keys[1]],  # year 2016
        feat[keys[2]],  # year 2017
        feat[keys[3]],  # year 2018
        feat[keys[4]],  # year 2019
        feat[keys[5]]   # year 2020
    ]).reset_index(drop=True)
    return df

### Convert to datetime

On peeking through the dataframes nested in the dictionaries `price` and `load`, the columns showing the time display them as time ranges e.g `31.12.2020 19:00 - 31.12.2020 20:00`. A function is created to make a new column called _time_ that will strip the string down to the initial timestamp e.g `31.12.2020 19:00` and then convert the series to datetime objects

In [9]:
def get_datetime(df):
    try:
        column = df['MTU (CET)'] # column with time values in price df
    except:
        column = df['Time (CET)']  # load time
    
    # create new column 'time' by formatting the original time column to get single timepoint instead of a range
    df['time'] = column.apply(lambda _: _[:16])
    # convert new time column from str to timestamp
    df['time'] = pd.to_datetime(df['time'])
        
    return df

### Merge datasets

In [10]:
def merge_data(feat_1, feat_2):
    df = feat_1.merge(feat_2, on='time').reset_index(drop=True)
    return df

## Final function

In [11]:
def fetch_data(path):
    
    price, load = get_features_df(main_path=path) # unpack tuple
    
    prices_data = concat_dataframes(price)
    load_data = concat_dataframes(load)

    load_data = get_datetime(load_data)
    prices_data = get_datetime(prices_data)

    # date up until
    idx_p = prices_data[prices_data['time'] == '2020-11-23 23:00:00'].index # valid time frame
    idx_l = load_data[load_data['time'] == '2020-11-23 23:00:00'].index
    assert(idx_p == idx_l)
    idx = idx_l[0] + 1
    
    df = merge_data(prices_data, load_data)
    df = df.iloc[:idx]

    # drop unused columns
    to_drop = ['MTU (CET)', 'Time (CET)', 'Day-ahead Total Load Forecast [MW] - BZN|DK1']
    df.drop(columns=to_drop, inplace=True)
    
    df = df.rename(columns={'Day-ahead Price [EUR/MWh]':'price',
                   'Actual Total Load [MW] - BZN|DK1':'load'})
    
    df = df[['time','price','load']] # reorder columns

    return df

### Test

In [12]:
df = fetch_data(path)

In [13]:
df.head(3)

,time,price,load
0,2015-01-01 00:00:00,25.02,1877
1,2015-01-01 01:00:00,18.29,1843
2,2015-01-01 02:00:00,16.04,1795
